In [2]:
# first call the data and make it data frame

import pandas as pd

file_path = r'G:\My Drive\AI 701 project\For Python\2022 training set_clean_good.xlsx'  
sheet_name = 'For Python'  

# Load the 2022 file
data22 = pd.read_excel(file_path, sheet_name=sheet_name)


tfile_path = r'G:\My Drive\AI 701 project\2023 for testing.xlsx'  
tsheet_name = 'Clean'  

# Load the 2023 file
data23 = pd.read_excel(tfile_path, sheet_name=tsheet_name)



In [3]:
#Pick the desired Columns
Selected = ['C1','Sulphur','NGL','Feed O ', 'Feed B', 'Feed S', 'Feed K','Feed I','Feed N']
data23_s = data23[Selected].copy()
data22_s = data22[Selected].copy()



In [4]:
#add in 2021 and 2024, and fix the data in a similar sense.
onefile_path = r'G:\My Drive\AI 701 project\2021 daily actuals clean.xlsx'  
onesheet_name = 'Clean'  

# Load the 2021 file
data21 = pd.read_excel(onefile_path, sheet_name=onesheet_name)



ffile_path = r'G:\My Drive\AI 701 project\2024 daily actuals clean.xlsx'  
fsheet_name = 'Clean'  

# Load the 2024 file
data24 = pd.read_excel(ffile_path, sheet_name=fsheet_name)

# some data manipulations to fit our model.


data24 = data24.iloc[:, :13] #remove plant X data
data21 = data21.iloc[:, :13] #remove plant X data

data24.columns = data24.iloc[0]  # Set the first row as the column names
data24 = data24[2:]  # Remove the first row as it's now the header, and 2nd row which is units

# Reset index if needed
data24.reset_index(drop=True, inplace=True)

data21.columns = data21.iloc[0]  # Set the first row as the column names
data21 = data21[2:]  # Remove the first row as it's now the header, and 2nd row which is units

# Reset index if needed
data21.reset_index(drop=True, inplace=True)

#rename the columns same as 2023 and 2023
data24.rename(columns={'NGL\n': 'NGL'}, inplace=True)
data24.rename(columns={'C1\n': 'C1'}, inplace=True)

data21.rename(columns={'NGL\n': 'NGL'}, inplace=True)
data21.rename(columns={'C1\n': 'C1'}, inplace=True)

data21.rename(columns={'Feed O': 'Feed O '}, inplace=True) #need to add the space
data24.rename(columns={'Feed O': 'Feed O '}, inplace=True)



In [5]:
data21_s = data21[Selected].copy()
data24_s = data24[Selected].copy()


In [6]:
#combine 22 and 23 for one big data #I now added 2021 and 2024

data1 = pd.concat([data22_s, data23_s, data21_s, data24_s], ignore_index=True)


In [7]:
import numpy as np

# Count NaN values
nan_count = data1.isna().sum().sum()  # Total NaNs
print(f"Total NaN values: {nan_count}")

# Count -ve
negative_count = (data1 < 0).sum().sum()  # Total zeros
print(f"Total -ve values: {negative_count}")

negative_per_column = (data1 < 0).sum()
print("Negative values per column:")
print(negative_per_column)

data1[data1 < 0] = 0
negative_count = (data1 < 0).sum().sum()  # Total zeros
print(f"Total -ve values: {negative_count}")



Total NaN values: 0
Total -ve values: 34
Negative values per column:
C1         27
Sulphur     0
NGL         6
Feed O      0
Feed B      0
Feed S      0
Feed K      0
Feed I      1
Feed N      0
dtype: int64
Total -ve values: 0


In [8]:
#Now for Each product, do the adjustment and then drop the negatives. (C1 is main culrpit)
# Known weights for C1 and Sulphur

c1_weights = np.array([0.84, 0.72, 0.84, 0.70])
s_weights = np.array([0.90, 1.55, 2, 0])
ngl_weights = np.array([2.49,7.8,2,9.47])

data1['C1_adj'] = data1['C1'] - np.dot(data1[['Feed I', 'Feed B', 'Feed S', 'Feed K']].values, c1_weights)
data1['Sulphur_adj'] = data1['Sulphur'] - np.dot(data1[['Feed I', 'Feed B', 'Feed S', 'Feed K']].values, s_weights)
data1['NGL_adj'] = data1['NGL'] - np.dot(data1[['Feed I', 'Feed B', 'Feed S', 'Feed K']].values, ngl_weights)

negative_per_column = (data1 < 0).sum()
print("Negative values per column:")
print(negative_per_column)


Negative values per column:
C1               0
Sulphur          0
NGL              0
Feed O           0
Feed B           0
Feed S           0
Feed K           0
Feed I           0
Feed N           0
C1_adj         392
Sulphur_adj    187
NGL_adj         32
dtype: int64


In [9]:
data2 = data1.drop(data1[data1['C1_adj'] < 0].index)
negative_per_c2= (data2 < 0).sum()
print("Negative values per column:")
print(negative_per_c2)

print(data2.shape)

Negative values per column:
C1              0
Sulphur         0
NGL             0
Feed O          0
Feed B          0
Feed S          0
Feed K          0
Feed I          0
Feed N          0
C1_adj          0
Sulphur_adj    82
NGL_adj         2
dtype: int64
(841, 12)


In [10]:
data2 = data2.drop(data2[data1['NGL_adj'] < 0].index)


C:\Users\Admin\AppData\Local\Temp\ipykernel_21636\759429870.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data2 = data2.drop(data2[data1['NGL_adj'] < 0].index)


In [11]:
from sklearn.model_selection import train_test_split

data2_train, data2_test = train_test_split (data2, test_size=0.2, random_state=0)


In [12]:
from sklearn.linear_model import LinearRegression
# Perform linear regression to find w5 and w6
x_train = data2_train[['Feed O ', 'Feed N']]
ny_train = data2_train[['NGL_adj']]
c1y_train = data2_train[['C1_adj']]
sy_train = data2_train[['Sulphur_adj']]
y_train = data2_train[['NGL_adj', 'C1_adj', 'Sulphur_adj']]


c1model = LinearRegression(fit_intercept=False, positive=True)
c1model.fit(x_train, c1y_train)

nmodel = LinearRegression()
nmodel.fit(x_train, ny_train)

smodel = LinearRegression(fit_intercept=False, positive=True)
smodel.fit(x_train, sy_train)



LinearRegression(fit_intercept=False, positive=True)

In [35]:
model = LinearRegression(fit_intercept=False, positive=True)
model.fit(x_train, y_train)

LinearRegression(fit_intercept=False, positive=True)

In [36]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score, root_mean_squared_error

def evaluate_model_performance(y_true, y_pred):
    # Calculate all metrics
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)  # RMSE is the square root of MSE
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100  # Convert MAPE to percentage
    r2 = r2_score(y_true, y_pred)

    # Print out the results
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
    print(f"R^2 Score: {r2:.4f}")

    # # Optionally, return results in a dictionary
    # return {
    #     "MAE": mae,
    #     "MSE": mse,
    #     "RMSE": rmse,
    #     "MAPE": mape,
    #     "R2": r2
    # }

# Usage

In [37]:
x_test = data2_test[['Feed O ', 'Feed N']]
ny_test = data2_test[['NGL_adj']]
c1y_test = data2_test[['C1_adj']]
sy_test = data2_test[['Sulphur_adj']]

y_test = data2_test[['NGL_adj', 'C1_adj', 'Sulphur_adj']]


In [38]:
#now to test the model : 
ny_pred = nmodel.predict(x_test)
c1y_pred = c1model.predict(x_test)
sy_pred = smodel.predict(x_test)
c1y_pred_train = c1model.predict(x_train)

y_pred = model.predict(x_test)

In [39]:
evaluate_model_performance(ny_test, ny_pred)

Mean Absolute Error (MAE): 1023.5767
Mean Squared Error (MSE): 1635248.1471
Root Mean Squared Error (RMSE): 1278.7682
Mean Absolute Percentage Error (MAPE): 17.63%
R^2 Score: 0.4139


In [40]:
evaluate_model_performance(c1y_test, c1y_pred)

Mean Absolute Error (MAE): 70.5887
Mean Squared Error (MSE): 7258.6820
Root Mean Squared Error (RMSE): 85.1979
Mean Absolute Percentage Error (MAPE): 255.33%
R^2 Score: -0.0399


In [41]:
c1model.coef_

array([[0.2907194 , 0.64545328]])

In [42]:
nmodel.coef_

array([[17.19259599, 18.96118399]])

In [43]:
smodel.coef_

array([[3.00731724, 0.9721383 ]])

In [44]:
evaluate_model_performance(y_test, y_pred)

Mean Absolute Error (MAE): 520.8661
Mean Squared Error (MSE): 667870.2196
Root Mean Squared Error (RMSE): 646.9484
Mean Absolute Percentage Error (MAPE): 204.73%
R^2 Score: 0.0968


In [45]:
model.coef_

array([[18.39041709, 22.10035814],
       [ 0.2907194 ,  0.64545328],
       [ 3.00731724,  0.9721383 ]])